In [1]:
import os
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, WebDriverException
from bs4 import BeautifulSoup as bs
import re
import requests
import time, random
import pandas as pd

chromedriver = '/usr/bin/chromedriver'
os.environ["webdriver.chrome.driver"] = \
    chromedriver
driver = webdriver.Chrome(chromedriver)
driver2 = webdriver.Chrome(chromedriver)

/tmp/ipykernel_241748/2628382612.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver)
/tmp/ipykernel_241748/2628382612.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver2 = webdriver.Chrome(chromedriver)


In [2]:
# scraping a single post from vanlifetrader

#driver.get('https://vanlifetrader.com/listing/outside-van-family-camper/')

In [3]:
#print(driver.page_source[:1000])

In [4]:
#soup = bs(driver.page_source)

#basics_table = soup.find_all(class_='col-md-12 block-type-table')[0]
#tech_table = soup.find_all(class_='col-md-12 block-type-table')[1]

In [5]:
''' I want to scrape info from the first table on the page (vehicle, price, location), the technical specs table, and the features table.
Two of these elements (top table, tech specs) are all referred to as <div class="col-md-12 block-type-table" id='block_#######'> 
The features table is referred to as <div class="col-md-12 block-type-table" id='block_#######'> 
The id "#######" for each is different for each element, but the syntax of "block_" is the same. Can I exploit this?'''

#basics = basics_table.find_all('li')
#tech_specs = tech_table.find_all('li')
#features = soup.find(class_='col-md-12 block-type-tags').find_all('span')

' I want to scrape info from the first table on the page (vehicle, price, location), the technical specs table, and the features table.\nTwo of these elements (top table, tech specs) are all referred to as <div class="col-md-12 block-type-table" id=\'block_#######\'> \nThe features table is referred to as <div class="col-md-12 block-type-table" id=\'block_#######\'> \nThe id "#######" for each is different for each element, but the syntax of "block_" is the same. Can I exploit this?'

In [6]:
#basics

In [7]:
'''
basics_dict = {}
post_dict = {}

for item in basics:
    post_dict[item.find(class_='item-attr').text] = item.find(class_='item-property').text

for k,v in post_dict.items():
    post_dict[k] = v.replace('\n','')
'''

"\nbasics_dict = {}\npost_dict = {}\n\nfor item in basics:\n    post_dict[item.find(class_='item-attr').text] = item.find(class_='item-property').text\n\nfor k,v in post_dict.items():\n    post_dict[k] = v.replace('\n','')\n"

In [8]:
#basics_dict

basics_vehicle = basics[0].text
asics_price = basics[1].text
basics_loc = basics[2].text

basics_vehicle = basics_vehicle.split()

basics_vehicle

basics_v_dict = {basics_vehicle[0]:str(basics_vehicle[1]) + " " + str(basics_vehicle[2]) + " " + str(basics_vehicle[3]) + " " + str(basics_vehicle[4])}

basics_v_dict

basics_price = basics_price.split()

basics_price

basics_price[1] = basics_price[1].replace("$","").replace(",","")

basics_p_dict = {basics_price[0]: int(basics_price[1])}

basics_loc = basics_loc.split()

basics_p_dict

basics_loc

basics_l_dict = {basics_loc[0]: str(basics_loc[1]) + " " + str(basics_loc[2]) + " " + str(basics_loc[3]) + " " + str(basics_loc[4])}

basics_l_dict

basics_list = [basics_v_dict, basics_p_dict, basics_l_dict]

basics_list

In [9]:
#features

In [10]:
'''
features_clean = []

for i in features:
    i = str(i)
    i = i.replace('<span>',"").replace('</span>',"")
    features_clean.append(i)
'''

'\nfeatures_clean = []\n\nfor i in features:\n    i = str(i)\n    i = i.replace(\'<span>\',"").replace(\'</span>\',"")\n    features_clean.append(i)\n'

In [11]:
#features_clean

In [12]:
#type(tech_specs)

In [13]:
#tech_specs[0]

In [14]:
#Can I use a loop create a dict, and then slice into each list item, and extract the text of "item-attr" to create the key and 'item-property' text as the value?

In [15]:
'''
tech_specs_dict = {}

for item in tech_specs:
    post_dict[item.find(class_='item-attr').text] = item.find(class_='item-property').text

for k,v in post_dict.items():
    post_dict[k] = v.replace('\n','')
'''

"\ntech_specs_dict = {}\n\nfor item in tech_specs:\n    post_dict[item.find(class_='item-attr').text] = item.find(class_='item-property').text\n\nfor k,v in post_dict.items():\n    post_dict[k] = v.replace('\n','')\n"

In [16]:
#tech_specs_dict

In [17]:
'''
features_dict = {}

for i in features_clean:
    post_dict[i] = True
'''

'\nfeatures_dict = {}\n\nfor i in features_clean:\n    post_dict[i] = True\n'

In [18]:
#post_dict

Alright, so that is maybe one post cleaned. I have a lot more features than I was expecting: can these be boolean values? 
True if the van has it, False if it doesn't.

I am concerned that, since not every van will have the same features, and the website only displays what it DOES have, I will find it
really hard to actually compare vans that DO have showers against those that DON'T. How do I deal with this?
*strategy: move into Pandas, leverage merge functionality to get NaN's that I can change to False values for features*
*master list of features I want to include that I can use to filter?*
**get_dummies can turn categorical values into booleans**

Can I clean up values that should be integers once I move this into a pandas df?

In [19]:
'''
regex = '^\(d+),\(d+)$'

for k,v in post_dict.items():
    if "$" in str(v):
        post_dict[k] = float(v.replace('$','').replace(',',''))
    elif regex in str(v):
        post_dict[k] = int(v.replace(',',''))
    elif "MPG" in str(v):
        post_dict[k] = float(v.replace('MPG',''))
    elif str(v).isdigit():
        post_dict[k] = int(v)
    else:
        continue
'''
#how do i get the regex to convert the mileage to an integer?

'\nregex = \'^\\(d+),\\(d+)$\'\n\nfor k,v in post_dict.items():\n    if "$" in str(v):\n        post_dict[k] = float(v.replace(\'$\',\'\').replace(\',\',\'\'))\n    elif regex in str(v):\n        post_dict[k] = int(v.replace(\',\',\'\'))\n    elif "MPG" in str(v):\n        post_dict[k] = float(v.replace(\'MPG\',\'\'))\n    elif str(v).isdigit():\n        post_dict[k] = int(v)\n    else:\n        continue\n'

In [20]:
#post_dict

In [21]:
#testing a second page to see if my search method works
'''
driver.get('https://vanlifetrader.com/listing/awesome-stealthy-adventure-van/')

soup2 = bs(driver.page_source)

basics_table2 = soup2.find_all(class_='col-md-12 block-type-table')[0]
tech_table2 = soup2.find_all(class_='col-md-12 block-type-table')[1]

basics2 = basics_table2.find_all('li')
tech_specs2 = tech_table2.find_all('li')
features2 = soup2.find(class_='col-md-12 block-type-tags').find_all('span')

post_dict2 = {}
features_clean2 = []

for item in basics2:
    post_dict2[item.find(class_='item-attr').text] = item.find(class_='item-property').text

for k,v in post_dict2.items():
    post_dict2[k] = v.replace('\n','')
    
for i in features2:
    i = str(i)
    i = i.replace('<span>',"").replace('</span>',"")
    features_clean2.append(i)
    
for item in tech_specs2:
    post_dict2[item.find(class_='item-attr').text] = item.find(class_='item-property').text

for k,v in post_dict2.items():
    post_dict2[k] = v.replace('\n','')

for i in features_clean2:
    post_dict2[i] = True
    
regex = '^\(d+),\(d+)$'

for k,v in post_dict2.items():
    if "$" in str(v):
        post_dict2[k] = float(v.replace('$','').replace(',',''))
    elif k == 'Mileage':
        post_dict2[k] = int(v.replace(',',''))
    elif "MPG" in str(v):
        post_dict2[k] = float(v.replace('MPG',''))
    elif str(v).isdigit():
        post_dict2[k] = int(v)
    else:
        continue
'''
#how do i get the regex to convert the mileage to an integer?

'\ndriver.get(\'https://vanlifetrader.com/listing/awesome-stealthy-adventure-van/\')\n\nsoup2 = bs(driver.page_source)\n\nbasics_table2 = soup2.find_all(class_=\'col-md-12 block-type-table\')[0]\ntech_table2 = soup2.find_all(class_=\'col-md-12 block-type-table\')[1]\n\nbasics2 = basics_table2.find_all(\'li\')\ntech_specs2 = tech_table2.find_all(\'li\')\nfeatures2 = soup2.find(class_=\'col-md-12 block-type-tags\').find_all(\'span\')\n\npost_dict2 = {}\nfeatures_clean2 = []\n\nfor item in basics2:\n    post_dict2[item.find(class_=\'item-attr\').text] = item.find(class_=\'item-property\').text\n\nfor k,v in post_dict2.items():\n    post_dict2[k] = v.replace(\'\n\',\'\')\n    \nfor i in features2:\n    i = str(i)\n    i = i.replace(\'<span>\',"").replace(\'</span>\',"")\n    features_clean2.append(i)\n    \nfor item in tech_specs2:\n    post_dict2[item.find(class_=\'item-attr\').text] = item.find(class_=\'item-property\').text\n\nfor k,v in post_dict2.items():\n    post_dict2[k] = v.replac

In [22]:
#post_dict

In [23]:
#post_dict2

Awesome! I am scraping the same containers on each page and getting the correctly varying information. Questions moving forward:
- How do I handle the fact that each dictionary is going to have a slightly different combination of keys?
- How do I automate the cleaning of the mileage?
- How do I get a list of links to each post on a single page? (class_=results-view.grid).find_all('a') perhaps?

In [24]:
#driver.get('https://vanlifetrader.com/explore/?sort=latest')

In [25]:
#soup3 = bs(driver.page_source)

In [26]:
#print(soup3.find_all(class_='lf-item lf-item-default'))

In [27]:
def get_posts_on_page(page):
    
    '''
    Inputs: a url (string).
    
    Returns: a list containing all the links on that VanlifeTrader page that lead to each posting on the page
    '''
    
    # connect the driver to the page of postings and give it 30 seconds to load
    driver.get(page)
    time.sleep(30)
    
    # create a BeautifulSoup object of the HTML source from the page, and pause for ten seconds
    soup = bs(driver.page_source)
    time.sleep(10)
    
    # initialize an empty list to collect the post links in
    posts_on_page = []

    # find the post links and append them to posts_on_page
    for div in soup.find_all(class_='lf-item lf-item-default'):
        url_tag = div.find('a')
        url = url_tag['href']
        posts_on_page.append(url)
        
    return posts_on_page

In [28]:
#test_url = 'https://vanlifetrader.com/explore/?pg=6&sort=latest'

#test_list = get_posts_on_page(test_url)

In [29]:
#len(test_list)

In [30]:
#lets try to scrape one page

In [31]:
def page_scraper(page_listings):

    posts = []

    # loop through every link in page_listings
    for url in page_listings:
        driver2.get(url) # connect to the specific posting page
        soup = bs(driver2.page_source) # bring the source HTML to BeautifulSoup
    
        basics_table = soup.find_all(class_='col-md-12 block-type-table')[0] # locate the "basics" element on the page
        tech_table = soup.find_all(class_='col-md-12 block-type-table')[1] # locate the "tech specs" element on the page
        features = soup.find(class_='col-md-12 block-type-tags').find_all('span') # locate the "features" element on the page
    
        post_dict = {} # create a new dictionary to hold the information from the post; this will be recreated every iteration of the outer loop
    
        basics = basics_table.find_all('li') # drill down to the list elements of the "basics" table
        tech_specs = tech_table.find_all('li') # drill down to the list elements of the "tech specs" table
        features = soup.find(class_='col-md-12 block-type-tags').find_all('span') # drill down to the "span" element of the features table
        
        try:
            # create k:v pairs in post_dict from "basics" data
            for item in basics: 
                post_dict[item.find(class_='item-attr').text] = item.find(class_='item-property').text 
            # create k:v pairs in post_dict from "tech specs" data
            for item in tech_specs:
                post_dict[item.find(class_='item-attr').text] = item.find(class_='item-property').text
    
            # isolate the actual text in each element of features, and then create k:v pairs in post_dict with True values
            for i in features:
                i = str(i)
                i = i.replace('<span>',"").replace('</span>',"")
                post_dict[i] = True
        except:
            continue

        # clean up data in post_dict    
        for k,v in post_dict.items():
            try:
                if '\n' in str(v):
                    post_dict[k] = v.replace('\n','')
                if k == 'Price':
                    post_dict[k] = float(v.replace('$','').replace(',',''))
                if k == 'Mileage':
                    post_dict[k] = int(v.replace(',',''))
                if k == 'Fuel Efficiency (Highway)':
                    post_dict[k] = float(v.replace('MPG',''))
                if k == 'Number of Seats with Seatbelts' or k == 'Sleeping Capacity' or k == 'Wheel Base Length':
                    post_dict[k] = int(v)
                else:
                    continue
            except:
                continue
                
        # append post dictionaries to a list (posts) until every post on the page is scraped
        if len(posts) != len(page_listings):
            posts.append(post_dict)
        else:
            break

        # introduce a random pause based on 3 seconds between scrapes (from vanlifetrader's robots.txt)    
        time.sleep(1+3*random.random())

    # store the page data as a pandas dataframe once all posts are scraped    
    #if len(posts) == len(page_listings):
        posts_df = pd.DataFrame(posts)
        
    return posts_df

In [32]:
#test_df = page_scraper(test_list)

In [33]:
#test_df.head()

In [34]:
#posts_df.head()

In [35]:
# YAYUHHHHHHHHHHHHH

# Okay so....now I scrape the whole site?

In [36]:
url_base = 'https://vanlifetrader.com/explore/?pg=' ## pg needs to go from 1 - 11
sort = '&sort=latest'

page_list = [url_base + str(i) + sort for i in range(1,12)]

In [41]:
page_list

['https://vanlifetrader.com/explore/?pg=1&sort=latest',
 'https://vanlifetrader.com/explore/?pg=2&sort=latest',
 'https://vanlifetrader.com/explore/?pg=3&sort=latest',
 'https://vanlifetrader.com/explore/?pg=4&sort=latest',
 'https://vanlifetrader.com/explore/?pg=5&sort=latest',
 'https://vanlifetrader.com/explore/?pg=6&sort=latest',
 'https://vanlifetrader.com/explore/?pg=7&sort=latest',
 'https://vanlifetrader.com/explore/?pg=8&sort=latest',
 'https://vanlifetrader.com/explore/?pg=9&sort=latest',
 'https://vanlifetrader.com/explore/?pg=10&sort=latest',
 'https://vanlifetrader.com/explore/?pg=11&sort=latest']

In [38]:
# reinitialize drivers, if needed

driver = webdriver.Chrome(chromedriver)
driver2 = webdriver.Chrome(chromedriver)

vans = pd.DataFrame()

for page in page_list:
    
    print(f'Scraping this page: {page}')
    post_links = get_posts_on_page(page)
    print(f'I scraped {len(post_links)} links. The first one is {post_links[0]} and the last one is {post_links[-1]}.')  
    vans = vans.append(page_scraper(post_links))

/tmp/ipykernel_241748/1279694086.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver)
/tmp/ipykernel_241748/1279694086.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver2 = webdriver.Chrome(chromedriver)


Scraping this page: https://vanlifetrader.com/explore/?pg=1&sort=latest
I scraped 50 links. The first one is https://vanlifetrader.com/listing/custom-made-pro-master-camper-van/ and the last one is https://vanlifetrader.com/listing/2021-promaster-luxury-van-shower-toilet/.
Scraping this page: https://vanlifetrader.com/explore/?pg=2&sort=latest
I scraped 50 links. The first one is https://vanlifetrader.com/listing/brand-new-promaster-camper-conversion/ and the last one is https://vanlifetrader.com/listing/2016-sprinter-4x4-adventure-vehicle/.
Scraping this page: https://vanlifetrader.com/explore/?pg=3&sort=latest
I scraped 50 links. The first one is https://vanlifetrader.com/listing/custom-stealth-camper-van-solar-powered-tiny-home-on-wheels/ and the last one is https://vanlifetrader.com/listing/2015-4x4-mercedes-sprinter-2500/.
Scraping this page: https://vanlifetrader.com/explore/?pg=4&sort=latest
I scraped 50 links. The first one is https://vanlifetrader.com/listing/sale-pending-2016

In [39]:
vans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 520 entries, 0 to 19
Data columns (total 56 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Vehicle                           520 non-null    object 
 1   Price                             520 non-null    float64
 2   Location                          520 non-null    object 
 3   Manufacturing Year                520 non-null    object 
 4   Make & Model                      520 non-null    object 
 5   Mileage                           520 non-null    object 
 6   Drive                             520 non-null    object 
 7   Title Status                      520 non-null    object 
 8   Transmission                      520 non-null    object 
 9   Fuel                              520 non-null    object 
 10  Wheel Base Length                 359 non-null    object 
 11  Number of Seats with Seatbelts    520 non-null    int64  
 12  Sleeping 

In [40]:
#vans.to_csv('vanlifetrader_data_oct28.csv')

In [42]:
vans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 520 entries, 0 to 19
Data columns (total 56 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Vehicle                           520 non-null    object 
 1   Price                             520 non-null    float64
 2   Location                          520 non-null    object 
 3   Manufacturing Year                520 non-null    object 
 4   Make & Model                      520 non-null    object 
 5   Mileage                           520 non-null    object 
 6   Drive                             520 non-null    object 
 7   Title Status                      520 non-null    object 
 8   Transmission                      520 non-null    object 
 9   Fuel                              520 non-null    object 
 10  Wheel Base Length                 359 non-null    object 
 11  Number of Seats with Seatbelts    520 non-null    int64  
 12  Sleeping 

In [44]:
#vans.to_csv('vanlifetrader_data_nov4.csv')